In [1]:
from typing import *

import torch 
import numpy as np 
import shutil 
import json 
import zipfile 
import io 
import pytorch_lightning as pl 
from scipy.sparse import csc_matrix 
from pathlib import Path 
import torch.nn.functional as F
from torchmetrics.functional import accuracy, precision, recall 
from pytorch_tabnet.tab_network import TabNet
import copy
import warnings

import sys, os 
sys.path.append('../src')

from torchmetrics.functional import *
import torchmetrics 

from data import *
from lightning_train import *
from model import *

from torchmetrics.functional.classification.stat_scores import _stat_scores_update, _stat_scores
from sklearn.metrics import classification_report
from bigcsv import to_h5ad 

In [9]:
pd.read_csv('../data/dental/labels_human_dental.tsv', sep=None)

/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/ipykernel_23989/2847047300.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  pd.read_csv('../data/dental/labels_human_dental.tsv', sep=None)


,cell_id,cluster,categorical_cell_type,cluster_celltype,sample_source,cell_type
0,human_germinectomy_1_one_AAACCTGTCAACTCTT-1,5,8,5 - pulp,human_germinectomy,8
1,human_germinectomy_1_one_AAACGGGCATTGCGGC-1,5,8,5 - pulp,human_germinectomy,8
2,human_germinectomy_1_one_AAACGGGGTGCCTGTG-1,13,8,13 - pulp,human_germinectomy,8
3,human_germinectomy_1_one_AAAGATGTCACTCTTA-1,9,8,9 - pulp,human_germinectomy,8
4,human_germinectomy_1_one_AAAGCAACAACCGCCA-1,15,6,15 - other immune cells,human_germinectomy,6
...,...,...,...,...,...,...
41668,No5_24_yo_healthy_retained_one_TTTGTTGGTCGTCATA-1,1,8,1 - pulp,No5_24_yo_healthy_retained,8
41669,No5_24_yo_healthy_retained_one_TTTGTTGGTCTGTGCG-1,7,3,7 - glial,No5_24_yo_healthy_retained,3
41670,No5_24_yo_healthy_retained_one_TTTGTTGGTTATGTCG-1,9,8,9 - pulp,No5_24_yo_healthy_retained,8
41671,No5_24_yo_healthy_retained_one_TTTGTTGTCGCAGTTA-1,9,8,9 - pulp,No5_24_yo_healthy_retained,8


In [3]:
mouse_labels = pd.read_csv('../data/benchmark/mouse_labels.csv')
human_labels = pd.read_csv('../data/benchmark/human_labels.csv')

/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/ipykernel_23989/3098124994.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  mouse_labels = pd.read_csv('../data/benchmark/mouse_labels.csv')


In [61]:
human_data = an.read_h5ad('../data/benchmark/human.h5ad')
mouse_data = an.read_h5ad('../data/benchmark/mouse_clipped.h5ad')

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Let's clean the cell type labels for human data since there are some NaN values

In [5]:
fixed_human = human_labels.dropna(subset=['subclass_label'])
fixed_human.index.name = 'cell'
fixed_human = fixed_human.reset_index(drop=False)

fixed_human.to_csv('../data/benchmark/human_labels_clean.csv', index=False)

In [6]:
pd.read_csv('../data/benchmark/human_labels_clean.csv')

,cell,sample_name,exp_component_name,specimen_type,cluster_color,cluster_order,cluster_label,class_color,class_order,class_label,...,cell_type_alt_alias_order,cell_type_alt_alias_label,cell_type_designation_color,cell_type_designation_order,cell_type_designation_label,external_donor_name_color,external_donor_name_order,external_donor_name_label,outlier_call,outlier_type
0,1,F2S4_160113_027_B01,LS-15005h_S02_E1-50,nucleus,#E170FE,32.0,Inh L2-5 VIP TOX2,#0066FF,4.0,GABAergic,...,32.0,NaN,#E170FE,32.0,Neuron 032,#3DCC3D,2,H200.1025,False,NaN
1,2,F2S4_160113_027_C01,LS-15005h_S03_E1-50,nucleus,#8E5864,2.0,Inh L1 LAMP5 GGT8P,#0066FF,4.0,GABAergic,...,2.0,NaN,#8E5864,2.0,Neuron 002,#3DCC3D,2,H200.1025,False,NaN
2,3,F2S4_160113_027_D01,LS-15005h_S04_E1-50,nucleus,#8B5862,1.0,Inh L1 LAMP5 NDNF,#0066FF,4.0,GABAergic,...,1.0,NaN,#8B5862,1.0,Neuron 001,#3DCC3D,2,H200.1025,False,NaN
3,4,F2S4_160113_027_E01,LS-15005h_S05_E1-50,nucleus,#CF6EC9,34.0,Inh L1-3 VIP ZNF322P1,#0066FF,4.0,GABAergic,...,34.0,NaN,#CF6EC9,34.0,Neuron 034,#3DCC3D,2,H200.1025,False,NaN
4,5,F2S4_160113_027_F01,LS-15005h_S06_E1-50,nucleus,#E693CE,35.0,Inh L3 VIP CBLN1,#0066FF,4.0,GABAergic,...,35.0,NaN,#E693CE,35.0,Neuron 035,#3DCC3D,2,H200.1025,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47427,49412,F2S4_190227_100_C01,SM-GE4QU_S187_E1-50,nucleus,#312E27,111.0,Astro L1-6 FGFR3 ETNPPL,#17994B,3.0,Non-neuronal,...,111.0,NaN,#312E27,111.0,Non-neuron 001,#3DCC3D,2,H200.1025,False,NaN
47428,49413,F2S4_190227_100_E01,SM-GE4QU_S189_E1-50,nucleus,#BFC124,75.0,Exc L6 THEMIS LINC00343,#AFCC3D,2.0,Glutamatergic,...,75.0,NaN,#BFC124,75.0,Neuron 075,#3DCC3D,2,H200.1025,False,NaN
47429,49414,F2S4_190227_100_F01,SM-GE4QU_S190_E1-50,nucleus,#8B5862,1.0,Inh L1 LAMP5 NDNF,#0066FF,4.0,GABAergic,...,1.0,NaN,#8B5862,1.0,Neuron 001,#3DCC3D,2,H200.1025,False,NaN
47430,49415,F2S4_190227_100_G01,SM-GE4QU_S191_E1-50,nucleus,#71AF9A,116.0,Oligo L4-6 OPALIN,#17994B,3.0,Non-neuronal,...,116.0,NaN,#71AF9A,116.0,Non-neuron 006,#3DCC3D,2,H200.1025,False,NaN


And again for the mouse labels

In [64]:
fixed_mouse = mouse_labels.dropna(subset=['subclass_label'])
fixed_mouse.index.name = 'cell'
fixed_mouse = fixed_mouse.reset_index(drop=False)

fixed_mouse.to_csv('../data/benchmark/mouse_labels_clean.csv', index=False)

In [65]:
mouse_labels['subclass_label'].isna().sum()

0

In [48]:
assert mouse_labels.shape[0] == mouse_data.shape[0]

In [49]:
assert human_labels.shape[0] == human_data.shape[0]

In [24]:
pd.read_csv(labelfiles[0]).loc[:, 'subclass_label'].unique()

array([nan, 'VIP', 'LAMP5', 'IT', 'PAX6', 'Oligodendrocyte', 'Astrocyte',
       'L5/6 IT Car3', 'L5/6 NP', 'SST', 'L6 CT', 'OPC', 'PVALB', 'L6b',
       'Microglia', 'L5 ET', 'Pericyte', 'Endothelial', 'L4 IT', 'VLMC'],
      dtype=object)

Since there are some extra sample in the mouse data, we'll take the intersection of the samples labeled and those in the entire dataset, and write it out to a new file.

In [13]:
# comb = list(set(mouse_data.obs['index'].values).intersection(mouse_labels['sample_name']))
# to_keep = mouse_data.obs[mouse_data.obs['index'].isin(comb)].index.astype(int)

# clipped = an.AnnData(
#     X=mouse_data.X[to_keep, :],
#     obs=mouse_data.obs[mouse_data.obs['index'].isin(comb)],
#     var=mouse_data.var
# )
# clipped.write('../data/benchmark/mouse_clipped.h5ad')

Let's look at which genes intersect between the two datasets

In [15]:
hg = [x.upper() for x in human_data.var.index]
mg = [x.upper() for x in mouse_data.var.index]

len(set(hg).intersection(mg))

16519